In [2]:
import sys
sys.path.append('../src')

In [3]:
import pandas as pd
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource, Scope
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from itertools import product
from energiapy.components.result import Result
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.plot.plot_results import CostY, CostX
from energiapy.components.location import Location
from energiapy.plot import plot_results, plot_scenario
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output
from energiapy.model.constraints.integer_cuts import constraint_block_integer_cut, constraint_block_integer_cut_min
from scipy import stats

In [4]:
scales = TemporalScale(discretization_list=[1, 24])

In [5]:
bigM = 10**7
smallM = 0.001

In [6]:
wind_pred = pd.read_csv('WindPred.csv')
demand_pred = pd.read_csv('DemandPred.csv')
demand_pred = demand_pred.set_index('Unnamed: 0')
wind_pred = wind_pred.set_index('Unnamed: 0')

In [7]:
Wind = Resource(name='Wind', cons_max=bigM, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW',
                 label='Power generated', demand=True, varying=[VaryingResource.DETERMINISTIC_DEMAND])

H2O = Resource(name='H2O', cons_max=bigM, basis='tons', label='Water')

Uranium = Resource(name='Uranium', cons_max=1000,
                   price=42.70/(250/2), basis='kg', label='Uranium')



In [8]:
LiI = Process(name='LiI', storage=Power, vopex=1302, capex=35164, fopex=0, prod_min=0,  prod_max=200,
              storage_loss=0.11,  label='Lithium-ion battery', basis='MW', store_max=bigM)

WF = Process(name='WF', conversion={Wind: -1/0.85, Power: 1}, vopex=990, capex=146200, fopex=0, prod_min=0,
             prod_max=70, label='Wind mill array', basis='MW', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])

ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1},
               capex=7988951, fopex=0.04*0.730, vopex=7988, prod_min=0, prod_max=bigM, label='Small modular reactors (SMRs)', block='power_gen')

In [9]:
process_set = {LiI, WF, ASMR}

In [10]:
houston = Location(name='HO', processes=process_set, capacity_factor={WF: wind_pred}, demand_factor={Power: demand_pred},
                   scales=scales, label='Houston', expenditure_scale_level=0, demand_scale_level=1, price_scale_level=1, capacity_scale_level=1, gwp_scale_level = 1)

In [11]:

scenario = Scenario(name='scenario_full', network=houston, scales=scales,  expenditure_scale_level=0, demand_scale_level=1, 
                    purchase_scale_level = 1, scheduling_scale_level=1, network_scale_level=0, capacity_scale_level=1, gwp_scale_level = 1,
                    label='full_case', demand={houston: {Power:60}})

In [12]:
milp_cost = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.DEMAND}, objective=Objective.COST, inventory_zero={houston: {(LiI, Power): 10}})


constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint demand
objective cost


In [13]:

results = solve(scenario=scenario, instance=milp_cost,
                solver='gurobi', saveformat='.pkl', name='det_1')


Set parameter QCPDual to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 564 rows, 566 columns and 1371 nonzeros
Model fingerprint: 0xccaa3880
Coefficient statistics:
  Matrix range     [4e-05, 8e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+07]
Presolve removed 444 rows and 443 columns
Presolve time: 0.00s
Presolved: 120 rows, 123 columns, 335 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.391037e+02   0.000000e+00      0s
     109    1.0784845e+07   0.000000e+00   0.000000e+00      0s

Solved in 109 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.078484524e+07


In [14]:
results.output['Cap_P']

{('HO', 'ASMR', 0): 0.0,
 ('HO', 'LiI', 0): 22.03648351515046,
 ('HO', 'LiI_discharge', 0): 200.0,
 ('HO', 'WF', 0): 59.63087763097866}

In [15]:
results.output['P']

{('HO', 'ASMR', 0, 0): 0.0,
 ('HO', 'ASMR', 0, 1): 0.0,
 ('HO', 'ASMR', 0, 2): 0.0,
 ('HO', 'ASMR', 0, 3): 0.0,
 ('HO', 'ASMR', 0, 4): 0.0,
 ('HO', 'ASMR', 0, 5): 0.0,
 ('HO', 'ASMR', 0, 6): 0.0,
 ('HO', 'ASMR', 0, 7): 0.0,
 ('HO', 'ASMR', 0, 8): 0.0,
 ('HO', 'ASMR', 0, 9): 0.0,
 ('HO', 'ASMR', 0, 10): 0.0,
 ('HO', 'ASMR', 0, 11): 0.0,
 ('HO', 'ASMR', 0, 12): 0.0,
 ('HO', 'ASMR', 0, 13): 0.0,
 ('HO', 'ASMR', 0, 14): 0.0,
 ('HO', 'ASMR', 0, 15): 0.0,
 ('HO', 'ASMR', 0, 16): 0.0,
 ('HO', 'ASMR', 0, 17): 0.0,
 ('HO', 'ASMR', 0, 18): 0.0,
 ('HO', 'ASMR', 0, 19): 0.0,
 ('HO', 'ASMR', 0, 20): 0.0,
 ('HO', 'ASMR', 0, 21): 0.0,
 ('HO', 'ASMR', 0, 22): 0.0,
 ('HO', 'ASMR', 0, 23): 0.0,
 ('HO', 'LiI', 0, 0): 22.036483515150465,
 ('HO', 'LiI', 0, 1): 22.03648351515046,
 ('HO', 'LiI', 0, 2): 22.03648351515046,
 ('HO', 'LiI', 0, 3): 22.03648351515046,
 ('HO', 'LiI', 0, 4): 22.03648351515046,
 ('HO', 'LiI', 0, 5): 16.841863502923943,
 ('HO', 'LiI', 0, 6): 11.443521175815988,
 ('HO', 'LiI', 0, 7): 6.